In [1]:
from sklearn.cluster import KMeans
import scipy.io as scio
import numpy as np
import scipy.signal as signal
import os

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats as stats

import eFC 
import VeFC_ex2fd as cal_VeFC

In [3]:
378*28//2

5292

### 输入输出和参数设置

In [2]:
# 输入数据路径
GROUP1_DATA2021_FOLDER = 'data2021/Class1-Parent'#亲子组
GROUP2_DATA2021_FOLDER = 'data2021/Class2-Stranger'#陌生人组
GROUP1_DATA2023_FOLDER = 'data2023/Class1-Parent'#亲子组
GROUP2_DATA2023_FOLDER = 'data2023/Class2-Stranger'#陌生人组
GROUP1_DATAALL_FOLDER = 'data_all/Class1-Parent'#亲子组
GROUP2_DATAALL_FOLDER = 'data_all/Class2-Stranger'#陌生人组

# 输入数据结构：
# 每个数据文件包含6种拼图场景，其中前面的1/2表示儿童拼或者成人拼，
# 后面的1/2/3表示单独/陪伴/合作,只关注3（合作）
sit_c1 = 'ca11' #儿童拼-单独
sit_a1 = 'ca21' #成人拼-单独
sit_c2 = 'ca12' #儿童拼-陪伴
sit_a2 = 'ca22' #成人拼-陪伴
sit_c3 = 'ca13' #儿童拼-合作
sit_a3 = 'ca23' #成人拼-合作
# 每种情况包含4个cell文件，分别对应4个频段：1-3，3-6，6-9，9-12
delta,theta,alpha,belta=0,1,2,3
freq_all=['delta','theta','alpha','belta']
# 每个cell文件的格式为通道数*时间点数*被试数
# 通道数: 其中1-28为儿童，29-56为成人，通过subc选择类型
# 28个通道及其对应的索引为：
channame_full = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','T7','T8','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_all = list(range(0,28))
# 26个通道及其对应的索引为：
channame_test = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_test = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

In [3]:
# 输出文件
# 输出数据所要导出到的Excel文件
output_excel='output/EFC_feature.xlsx'
# df=pd.DataFrame() #构造原始数据文件
# df.to_excel(output_excel)

In [4]:
# 参数设置
chan_num = 28 #通道数
edge_num = int(chan_num*(chan_num-1)/2) #边数
Nuptran_indices = cal_VeFC.uptran_mask(chan_num) #矩阵上三角列向量的位置
Euptran_indices = cal_VeFC.uptran_mask(edge_num) #矩阵上三角列向量的位置
Vtype='all'

# itype指边连接强度的计算方式，'p','c','d'分别代表：PLV复指数、corr相关、delta做差
itype_all=['e','c','d']
itype=itype_all[0]
# corr_type指边与边之间共波动特性的计算方式，'cos','pearson','delta'分别代表：
# 余弦相似度，皮尔逊相关系数，直接做差
corr_type_all=['cos','pearson','delta']
corr_type=corr_type_all[0]

sit=sit_c3 #拼图场景
freq=alpha #频段
subc_all=['child','adult']
subc=subc_all[0]  #对象类型 

### 特征1：研究共节点的两条边对应的EFC值是否高于不共节点的两条边

In [41]:
# 导入mat数据
path_mat='data_all/Class1-Parent/2023052501_prt.mat'
data_mat = scio.loadmat(path_mat)
# data_mat的格式为情况*1*频段

# 选择目标模态对应的数据，ts的格式为被试数*时间点数*通道数
# 目标模态设置为：
ts = data_mat[sit][0][freq].T

# 群体subc选择
if subc=='child': # 如果subject的类型为 child
    ts_subc = ts[:,:,0:28]
elif subc=='adult': # 如果subject的类型为 adult
    ts_subc = ts[:,:,28:56]

# 被试组选择
i=0
E,edges,nodes1,nodes2 = eFC.Int_exp(ts_subc[i])
eFC_mat = eFC.eFC_mat_cs(E)
nodes1.shape

(378,)

In [42]:
def NB_Diff_trl(eFC_mat,edge_num,nodes1,nodes2):
    eFC_nbT=[]
    eFC_nbF=[]

    for i in range(edge_num):
        for j in range(edge_num):
        # 两条边不相同
            if i!=j:
            # 设置“是否有相同的结点”的flag标记nb
                nb=False
            # 比较两条边所有结点的编号
                if nodes1[i]==nodes2[i]:
                    nb=True
                elif nodes1[i]==nodes2[j]:
                    nb=True
                elif nodes1[j]==nodes2[i]:
                    nb=True
                elif nodes1[j]==nodes2[j]:
                    nb=True
            # 如果这两条边有相同的结点
                if nb==True:
                    eFC_nbT=np.append(eFC_nbT,eFC_mat[i,j])
            # 如果这两条边没有相同的结点
                else:
                    eFC_nbF=np.append(eFC_nbF,eFC_mat[i,j])
    # 将所有的EFC数值取绝对值：因为负相关也代表了一种相关性
    eFC_nbT_abs = np.abs(eFC_nbT)
    eFC_nbF_abs = np.abs(eFC_nbF)
    # 计算EFC矩阵中共节点的边对于数值的平均值
    eFC_nbT_mean = np.mean(eFC_nbT_abs)
    # 计算EFC矩阵中不共节点的边对于数值的平均值
    eFC_nbF_mean = np.mean(eFC_nbF_abs)
    
    return eFC_nbT_mean,eFC_nbF_mean

In [43]:
def NB_diff_dyad(ts):
    eFC_nbT_trl=[] # 共有结点的边的EFC平均值，所有trial汇总
    eFC_nbF_trl=[] # 不共有结点的边的EFC平均值，所有trial汇总
    for i in range(ts.shape[0]):
        if itype=='e':
            E = eFC.Int_exp(ts[i])[0]
        elif itype=='c':
            E = eFC.Int_delta(ts[i])[0]
        elif itype=='d':
            E = eFC.Int_delta(ts[i])[0]
        nodes1,nodes2 = eFC.Int_exp(ts[i])[2],eFC.Int_exp(ts[i])[3]
    # 选择相关系数的类型
        if corr_type=='cos':
            eFC_mat = eFC.eFC_mat_cs(E)
        elif corr_type=='pearson':
            eFC_mat = eFC.eFC_mat_pr(E)
        elif corr_type=='delta':
            eFC_mat = eFC.eFC_mat_delta(E)
    # 计算一个trial（对应一个EFC矩阵）的eFC_nbT_mean,eFC_nbF_mean
        eFC_nbT,eFC_nbF = NB_Diff_trl(eFC_mat,edge_num,nodes1,nodes2)
        eFC_nbT_trl=np.append(eFC_nbT_trl,eFC_nbT)
        eFC_nbF_trl=np.append(eFC_nbF_trl,eFC_nbF)
    # 所有trial的eFC_nbT、eFC_nbF取平均
    eFC_nbT_mean=np.mean(eFC_nbT_trl)
    eFC_nbF_mean=np.mean(eFC_nbF_trl)

    return eFC_nbT_trl,eFC_nbF_trl,eFC_nbT_mean,eFC_nbF_mean

In [44]:
def NB_diff_all(GROUP_DATA_FOLDER,sit,freq,subc):
    eFC_nbT_all=[] # 共有结点的边的EFC平均值，所有dyad汇总
    eFC_nbF_all=[] # 不共有结点的边的EFC平均值，所有dyad汇总
    for filepath,dirnames,filenames in os.walk(GROUP_DATA_FOLDER):
            for filename in filenames:
                path_mat = os.path.join(filepath,filename)
                # 数据为.mat格式 
                if path_mat[-4:] == '.mat':
                    # 导入mat数据
                    data_mat = scio.loadmat(path_mat)# data_mat的格式为情况*1*频段

                    # 选择目标模态对应的数据，ts的格式为被试数*时间点数*通道数
                    ts = data_mat[sit][0][freq].T
                    # 群体subc选择
                    if subc=='child': # 如果subject的类型为 child
                        ts_subc = ts[:,:,0:28]
                    elif subc=='adult': # 如果subject的类型为 adult
                        ts_subc = ts[:,:,28:56]
                    eFC_nbT_trl,eFC_nbF_trl,eFC_nbT_mean,eFC_nbF_mean = NB_diff_dyad(ts_subc)
                    eFC_nbT_all=np.append(eFC_nbT_all,eFC_nbT_mean)
                    eFC_nbF_all=np.append(eFC_nbF_all,eFC_nbF_mean)

    return eFC_nbT_all,eFC_nbF_all

In [45]:
eFC_nbT_all,eFC_nbF_all = NB_diff_all(GROUP1_DATAALL_FOLDER,sit,freq,subc)

In [46]:
#perform the paired samples t-test
stats.ttest_rel(eFC_nbT_all,eFC_nbF_all)

TtestResult(statistic=50.282168953705394, pvalue=1.7591371984595823e-44, df=50)

### 导出输出数据

In [47]:
# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = sit+'-'+subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl')
'''df = pd.DataFrame({new_sheet_name: []})
df.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)
excel_writer.save()'''

'df = pd.DataFrame({new_sheet_name: []})\ndf.to_excel(excel_writer, sheet_name=new_sheet_name, index=False)\nexcel_writer.save()'

In [48]:
# 拼接两组EFC数据
eFC_all = np.c_[eFC_nbT_all,eFC_nbF_all]

# 转换为DataFrame格式
df_eFC_all = pd.DataFrame(eFC_all)
# 设置列名
df_eFC_all.columns=['shared-node','no shared-node']

# 保存至excel
df_eFC_all.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer.save()

C:\Users\吴诚成\AppData\Local\Temp\ipykernel_1364\523881956.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()


### 其他特征

In [49]:
# 记录所有双边组的标记
# 两个数字分别表示两条边（在所有边中的排序）
edges12=[]

for i in range(edge_num):
    for j in range(edge_num):
    # # 两条边不相同
    #     if i!=j:
        # 用来记录一组双边中两条边的顺序
        edge12 = str(i)+"-"+str(j)
        edges12=np.append(edges12,edge12)
    
edges12

array(['0-0', '0-1', '0-2', ..., '377-375', '377-376', '377-377'],
      dtype='<U32')

In [50]:
edges_upt=edges12[Euptran_indices]
edges_upt.shape

(71253,)

In [51]:
# 具有显著高或者低的EFC数值的双边
def Dedge_dyad(ts):
    VeFC_trl=[] # 所有trial的VeFC向量汇总
    tn = 0
    for i in range(ts.shape[0]):
        if itype=='e':
            E = eFC.Int_exp(ts[i])[0]
        elif itype=='c':
            E = eFC.Int_delta(ts[i])[0]
        elif itype=='d':
            E = eFC.Int_delta(ts[i])[0]
    # 选择相关系数的类型
        if corr_type=='cos':
            eFC_mat = eFC.eFC_mat_cs(E)
        elif corr_type=='pearson':
            eFC_mat = eFC.eFC_mat_pr(E)
        elif corr_type=='delta':
            eFC_mat = eFC.eFC_mat_delta(E)
    # 将eFC_mat打平成向量，然后提取其中的上三角部分
        eFC_flatten = eFC_mat.flatten("F")
        VeFC_upt = eFC_flatten[Euptran_indices]
        if tn==0:
            VeFC_trl=VeFC_upt
            tn=1
        else:
            VeFC_trl=np.c_[VeFC_trl,VeFC_upt]
    # 所有trial的VeFC取平均
    VeFC_trl_mean=np.mean(VeFC_trl,axis=1)

    return VeFC_trl_mean

VeFC_trl_mean = Dedge_dyad(ts_subc)
VeFC_trl_mean.shape

(71253,)

In [52]:
VeFC_trl_mean.argsort

<function ndarray.argsort>

In [53]:
edges_upt[np.where(VeFC_trl_mean>0.8)]

array(['1-0', '2-0', '2-1', '7-6', '11-10', '16-15', '79-78', '104-87',
       '111-51', '118-88', '137-136', '172-171', '186-181', '187-135',
       '257-214', '263-220', '266-223', '267-224', '268-225', '270-227',
       '271-228', '272-229', '274-228', '275-252', '294-249', '298-0',
       '299-275', '299-297', '324-299', '331-326', '340-335', '348-346',
       '350-322', '350-324', '351-0', '357-306', '377-0', '377-350',
       '377-351'], dtype='<U32')

In [54]:
# 导入mat数据
path_mat='data/Class1-Parent/2021060701epoch(1-3-6-9-12)_noref.mat'
data_mat = scio.loadmat(path_mat)
# data_mat的格式为情况*1*频段

# 选择目标模态对应的数据，ts的格式为被试数*时间点数*通道数
# 目标模态设置为：
sit,freq,subc=sit_c3,theta,'child'
ts = data_mat[sit][0][freq].T

# 群体subc选择
if subc=='child': # 如果subject的类型为 child
    ts_subc = ts[:,:,0:28]
elif subc=='adult': # 如果subject的类型为 adult
    ts_subc = ts[:,:,28:56]

# 被试组选择
i=1
E,edges,nodes1,nodes2 = eFC.Int_exp(ts_subc[i])
eFC2 = eFC.eFC_mat(E)
eFC2.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/Class1-Parent/2021060701epoch(1-3-6-9-12)_noref.mat'

In [ ]:
# 导入mat数据
path_mat='data/Class1-Parent/2021061003epoch(1-3-6-9-12)_noref.mat'
data_mat = scio.loadmat(path_mat)
# data_mat的格式为情况*1*频段

# 选择目标模态对应的数据，ts的格式为被试数*时间点数*通道数
# 目标模态设置为：
sit,freq,subc=sit_c3,theta,'child'
ts = data_mat[sit][0][freq].T

# 群体subc选择
if subc=='child': # 如果subject的类型为 child
    ts_subc = ts[:,:,0:28]
elif subc=='adult': # 如果subject的类型为 adult
    ts_subc = ts[:,:,28:56]

# 被试组选择
i=0
E,edges,nodes1,nodes2 = eFC.Int_exp(ts_subc[i])
eFC3 = eFC.eFC_mat(E)
eFC3.shape

In [ ]:
data=pd.DataFrame(eFC1)
 
#绘制热度图：
plot=sns.heatmap(data=data,cmap="RdBu_r")

extnt = 10 #间隔数
plt.xticks(np.arange(0,eFC3.shape[0],extnt),nodes1[np.arange(0,eFC3.shape[0],extnt)])

plt.show()

In [ ]:
data=pd.DataFrame(eFC2)
 
#绘制热度图：
plot=sns.heatmap(data=data,cmap="RdBu_r")
 
plt.show()

In [ ]:
data=pd.DataFrame(eFC3)
 
#绘制热度图：
plot=sns.heatmap(data=data,cmap="RdBu_r")
 
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=100, random_state=0, n_init=10).fit(eFC1)

In [ ]:
print(kmeans.labels_)

In [ ]:
from sklearn.cluster import MeanShift

means = MeanShift()
# 模型拟合与聚类预测
label_EFC = means.fit_predict(eFC1)

In [ ]:
label_EFC